In [25]:
# import libraries
import streamlit as st
import pandas as pd
import numpy as np

import streamlit.components.v1 as components

from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.transports.server import ServerTransport


from typing import Any, List

In [17]:
import os
TOKEN_ID = os.environ.get('SPECKLE_TOKEN')

In [32]:
HOST = "https://speckle.xyz"
STREAM_ID = "0b7b9e7705"

def get_authenticated_client() -> SpeckleClient:
    """
    This assumes you already have a local account. If you don't already have one, you'll need
    to download the Speckle Manager and add an account for your server (which can be localhost).
    See the docs for more info: https://speckle.guide/user/manager.html
    """
    client = SpeckleClient(host=HOST)
    client.authenticate(token=TOKEN_ID)
    
    # account = get_default_account()
    # client.authenticate(token=TOKEN_ID)

    return client

def get_branches(client: SpeckleClient, stream_id: str = STREAM_ID) -> Any:
    # get the specified commit data
    stream = client.stream.get(STREAM_ID)
    # create an authenticated server transport from the client and receive the commit obj
    transport = ServerTransport(STREAM_ID, client)
    # get all the branches
    branches = client.branch.list(stream_id=STREAM_ID, branches_limit= 100)
    
    return branches

# def receive_data(client: SpeckleClient, stream_id: str = STREAM_ID, commit_id: str = COMMIT_ID) -> Any:
#     transport = ServerTransport(client, stream_id)

#     commit = client.commit.get(stream_id, commit_id)
#     data = operations.receive(commit.referencedObject, transport)

#     return data["data"]

def speckle_df(branches) -> pd.DataFrame:
    a = [(b.name, b.id) for b in branches]
    # create pandas dataframe
    df = pd.DataFrame(a, columns=['branch', 'id'])
    df = df.join(df.branch.str.split('/', expand=True))
    df = df.rename(columns={0:'revision', 1:'area',2:'metric'}, inplace=False)
    
    df = df[df['revision'] != 'main']
    
    return df

In [33]:

# create and authenticate a client
client = get_authenticated_client()
branches = get_branches(client = client, stream_id= STREAM_ID )
df_branches = speckle_df(branches)

# df_data # TODO 

2022-02-11 14:02:12.195 INFO    gql.transport.requests: >>> {"query": "query Server {\n  serverInfo {\n    name\n    company\n    description\n    adminContact\n    canonicalUrl\n    version\n    roles {\n      name\n      description\n      resourceTarget\n    }\n    scopes {\n      name\n      description\n    }\n    authStrategies {\n      id\n      name\n      icon\n    }\n  }\n}"}
2022-02-11 14:02:12.304 INFO    gql.transport.requests: <<< {"data":{"serverInfo":{"name":"Speckle XYZ","company":"AEC Systems Ltd.","description":"The Official Speckle Server - Empowering Your Design and Construction Data.","adminContact":"office@speckle.systems","canonicalUrl":null,"version":"2.3.13","roles":[{"name":"stream:owner","description":"Owners have full access, including deletion rights & access control.","resourceTarget":"streams"},{"name":"stream:contributor","description":"Contributors can create new branches and commits, but they cannot edit stream details or manage collaborators.","resou

In [34]:
df_branches #use df in the application

,branch,id,revision,area,metric
1,rev1/cfd/0,1e986514a3,rev1,cfd,0
2,rev1/cfd/45,35fde25a77,rev1,cfd,45
3,rev1/cfd/90,2f628a5b63,rev1,cfd,90
4,rev1/cfd/135,a9df7cf439,rev1,cfd,135
5,rev1/cfd/180,f5832180f1,rev1,cfd,180
6,rev1/cfd/225,057325d70d,rev1,cfd,225
7,rev1/cfd/270,5b77d22cfd,rev1,cfd,270
8,rev1/cfd/315,84164c4fdb,rev1,cfd,315
9,rev1/rad/summer,dc6677a6c5,rev1,rad,summer
10,rev1/rad/winter,a21f8e2601,rev1,rad,winter


## Here I want to explore how to get the data from the objects
Maybe it's better to ask in the forum if I can filter the date using a GraphAPI

In [204]:
all_data=[]
for b in branches: #branch
    commit = b.commits.items[0] #select lastest commit
    data = operations.receive(commit.referencedObject, transport)
    all_data.append((b.name,data))

# commit = client.commit.get(STREAM_ID, '2dd0e86d1a')
# data = operations.receive(commit.referencedObject, transport)
# data

[('main',
  Base(id: 32000fb67fbd4836d98f30989a7cbf11, speckle_type: Base, totalChildrenCount: 0)),
 ('rev1/cfd/0',
  Base(id: 168907004e280bad25e155818912a917, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/45',
  Base(id: c8360a94899982cbdf588be85c55e07f, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/90',
  Base(id: 9baa9ee9cd0e995635a147e6488f92f0, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/135',
  Base(id: 83fd7f320f0e7e0793e038aba95a6576, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/180',
  Base(id: 3b95271e6684d958025b1fe1884e236c, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/225',
  Base(id: 850696a01cc38ed8c13da8b0365d0c79, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/270',
  Base(id: 777e28ebca45d66030986a0df7199559, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/cfd/315',
  Base(id: 6d34927bb7ae8b1db01b71b38c8a5905, speckle_type: Base, totalChildrenCount: 12)),
 ('rev1/rad/summer',
  Base(id: 

In [242]:
dfs = []
meshes = []
for name, base in all_data:
    if "main" not in name:
        print (name)
        try:   
            context, results = base["@data"][0]
            print(type(context))
            meshes.append(context["@context"])
            df = pd.DataFrame(data["@results"], columns=[name])
            dfs.append(df)
        except:
            meshes.append(None)
            dfs.append(None)

rev1/cfd/0
rev1/cfd/45
rev1/cfd/90
rev1/cfd/135
rev1/cfd/180
rev1/cfd/225
rev1/cfd/270
rev1/cfd/315
rev1/rad/summer
<class 'specklepy.objects.base.Base'>
rev1/rad/winter
rev1/rad/spring
rev1/rad/annual
rev1/aq/adms
<class 'specklepy.objects.base.Base'>
rev1/daylight/da
rev1/daylight/df
rev1/daylight/udi
rev1/daylight/vsc
rev1/utci/summer
rev1/utci/spring
rev1/utci/winter
rev1/utci/annual
rev1/mrt/annual
rev1/mrt/spring
rev1/mrt/winter
mrt/summer
rev2/daylight/df


In [248]:
d1 = all_data[1][1]
d1["@data"][0][0]["@results"]

Mesh(id: 7b75401e1ad82d0e6f9810215aeb0ada, speckle_type: Objects.Geometry.Mesh, totalChildrenCount: 0)

In [249]:
d1 = all_data[2][1]
d1["@data"][0][0]["@results"]

Mesh(id: 824cd56fe47555a7305dbfc9e475f743, speckle_type: Objects.Geometry.Mesh, totalChildrenCount: 0)

In [256]:
d1 = all_data[10][1]
d = d1["@data"][0][0]
d["@results"]

Mesh(id: 97de3d1c0f1cd4bc52f19b2c45b37de8, speckle_type: Objects.Geometry.Mesh, totalChildrenCount: 0)

In [194]:
import pandas as pd
df = pd.DataFrame(data["@results"], columns=['results'])
df

,results
0,2.394314
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
11497,5.395884
11498,5.872495
11499,6.072322
11500,6.031944


In [182]:
base["@results"]["@results"]

[256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.7546068675497,
 256.75460686754946,
 256.7546068675496,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.75460686754946,
 256.7546068675

In [ ]:
results['rev1']['cfd'][0]

In [ ]:


# get the specified commit data
commit = client.commit.get(STREAM_ID, COMMIT_ID)

# create an authenticated server transport from the client and receive the commit obj
transport = ServerTransport(STREAM_ID, client)
res = operations.receive(commit.referencedObject, transport)

# get the list of levels from the received object
# levels = res["data"]

base = res["@data"][0][0][0][0][0]
base

df_values = base["@DF_values"]

df = pd.DataFrame(df_values, columns=['df_values'])

print(df.head())